In [13]:
pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.3/261.3 kB 19.4 MB/s eta 0:00:00


In [23]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

class Song:
    def __init__(self, title, artist, duration):
        self.title = title
        self.artist = artist
        self.duration = duration  # Duration in seconds

    def __str__(self):
        return f"{self.title} by {self.artist} ({self.duration // 60}m {self.duration % 60}s)"

class MusicStreamingService:
    def __init__(self):
        self.subscribed_genres = set()
        self.playlists = {}
        self.sp = self.authenticate_spotify()  # Initialize Spotify client first
        self.available_genres = self.get_available_genres()  # Then fetch available genres

    def authenticate_spotify(self):
        client_id = '91181b82ba804f3b8e39f97a841ff1aa'
        client_secret = 'e7ec68f240f2453abbd604d48550080c'
        credentials = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
        return spotipy.Spotify(client_credentials_manager=credentials)

    def get_available_genres(self):
        # Fetch available genres dynamically from Spotify
        recommendations = self.sp.recommendation_genre_seeds()
        return recommendations['genres']

    def browse_genres(self):
        print("\nAvailable genres:")
        for genre in self.available_genres:
            print(f"- {genre}")
        print()  # Extra space for clarity

    def browse_songs_in_genre(self, genre):
        print(f"\nSongs available in {genre} genre:")
        songs = self.get_songs_by_genre(genre)
        if not songs:
            print("No songs available in this genre.")
        else:
            for song in songs:
                print(song)
        print()  # Extra space for clarity

    def subscribe(self, genre):
        if genre in self.available_genres:
            self.subscribed_genres.add(genre)
            print(f"Subscribed to {genre} genre.\n")
        else:
            print("Genre not available.\n")

    def view_subscriptions(self):
        print("\nSubscribed genres:")
        for genre in self.subscribed_genres:
            print(f"- {genre}")
        print()  # Extra space for clarity

    def discover_music(self):
        if not self.subscribed_genres:
            print("Please subscribe to a genre first.\n")
            return

        print("Discover songs from subscribed genres:")
        for genre in self.subscribed_genres:
            print(f"\nGenre: {genre}")
            songs = self.get_songs_by_genre(genre)
            for song in songs:
                print(song)
        print()  # Extra space for clarity

    def get_songs_by_genre(self, genre):
        results = self.sp.search(q=f'genre:"{genre}"', type='track', limit=10)
        songs = []
        for item in results['tracks']['items']:
            title = item['name']
            artist = item['artists'][0]['name']
            duration = item['duration_ms'] // 1000  # Convert to seconds
            songs.append(Song(title, artist, duration))
        return songs

    def song_exists(self, title, artist):
        results = self.sp.search(q=f'track:"{title}" artist:"{artist}"', type='track', limit=1)
        return len(results['tracks']['items']) > 0

    def get_song_duration(self, title, artist):
        results = self.sp.search(q=f'track:"{title}" artist:"{artist}"', type='track', limit=1)
        if results['tracks']['items']:
            return results['tracks']['items'][0]['duration_ms'] // 1000  # Return duration in seconds
        return None  # Song not found

    def play_song(self, song):
        if self.song_exists(song.title, song.artist):
            print(f"Playing: {song}\n")
        else:
            print("Error: Unable to play the song. Song does not exist.\n")

    def play_all_songs_in_genre(self, genre):
        if genre not in self.subscribed_genres:
            print("Error: You must subscribe to the genre first.\n")
            return

        songs = self.get_songs_by_genre(genre)
        if not songs:
            print("No songs available in this genre to play.\n")
            return

        print(f"Playing all songs in {genre} genre:\n")
        for song in songs:
            self.play_song(song)

    def create_playlist(self, playlist_name):
        self.playlists[playlist_name] = []
        print(f"Playlist '{playlist_name}' created.\n")

    def add_to_playlist(self, playlist_name, song):
        if playlist_name in self.playlists:
            if self.song_exists(song.title, song.artist):
                duration = self.get_song_duration(song.title, song.artist)
                if duration is not None:
                    song.duration = duration  # Update duration with actual duration from Spotify
                    self.playlists[playlist_name].append(song)
                    print(f"Added {song} to playlist '{playlist_name}'.\n")
                else:
                    print("Error: Unable to retrieve the song duration. Cannot add to playlist.\n")
            else:
                print("Error: Song does not exist. Cannot add to playlist.\n")
        else:
            print("Error: Playlist not found.\n")

    def search_song(self, search_type, query):
        if search_type == 'title':
            results = self.sp.search(q=f'track:"{query}"', type='track', limit=10)
        elif search_type == 'artist':
            results = self.sp.search(q=f'artist:"{query}"', type='track', limit=10)
        else:  # Search by both title and artist
            results = self.sp.search(q=f'track:"{query}"', type='track', limit=10)

        if not results['tracks']['items']:
            print("No songs found for your search.\n")
            return

        print("Search results:")
        for item in results['tracks']['items']:
            title = item['name']
            artist = item['artists'][0]['name']
            duration = item['duration_ms'] // 1000  # Convert to seconds
            print(f"{title} by {artist} ({duration // 60}m {duration % 60}s)")
        print()  # Extra space for clarity

    def play_songs_from_playlist(self, playlist_name):
        if playlist_name not in self.playlists:
            print("Error: Playlist not found.\n")
            return

        print(f"Playing all songs from playlist '{playlist_name}':\n")
        for song in self.playlists[playlist_name]:
            self.play_song(song)

# Interactive console for user input
def main():
    service = MusicStreamingService()

    while True:
        print("\nMusic Streaming Service")
        print("1. Browse genres")
        print("2. Browse songs in a genre")
        print("3. Subscribe to a genre")
        print("4. View subscriptions")
        print("5. Discover music")
        print("6. Create playlist")
        print("7. Add song to playlist")
        print("8. Search for a song")
        print("9. Play a song")
        print("10. Play all songs in a genre")
        print("11. Play all songs from a playlist")
        print("12. Exit")

        choice = input("Choose an option: ")
        if choice == '1':
            service.browse_genres()
        elif choice == '2':
            genre = input("Enter genre to browse songs: ").strip().lower()
            if genre in service.available_genres:
                service.browse_songs_in_genre(genre)
            else:
                print("Genre not available.\n")
        elif choice == '3':
            genre = input("Enter genre to subscribe: ").strip().lower()
            service.subscribe(genre)
        elif choice == '4':
            service.view_subscriptions()
        elif choice == '5':
            service.discover_music()
        elif choice == '6':
            playlist_name = input("Enter playlist name: ").strip()
            service.create_playlist(playlist_name)
        elif choice == '7':
            playlist_name = input("Enter playlist name: ").strip()
            song_title = input("Enter song title: ").strip()
            artist_name = input("Enter artist name: ").strip()
            song = Song(song_title, artist_name, 0)  # Duration will be fetched from Spotify
            service.add_to_playlist(playlist_name, song)
        elif choice == '8':
            search_option = input("Search by (1) Title, (2) Artist, (3) Both: ").strip()
            query = input("Enter your search query: ").strip()

            if search_option == '1':
                service.search_song('title', query)
            elif search_option == '2':
                service.search_song('artist', query)
            elif search_option == '3':
                service.search_song('both', query)
            else:
                print("Invalid option. Please try again.\n")
        elif choice == '9':
            song_title = input("Enter the title of the song to play: ").strip()
            artist_name = input("Enter the artist of the song: ").strip()
            duration = service.get_song_duration(song_title, artist_name)
            if duration is not None:
                song = Song(song_title, artist_name, duration)
                service.play_song(song)
            else:
                print("Error: Unable to play the song. Song does not exist.\n")
        elif choice == '10':
            genre = input("Enter the genre to play all songs: ").strip().lower()
            service.play_all_songs_in_genre(genre)
        elif choice == '11':
            playlist_name = input("Enter the name of the playlist to play: ").strip()
            service.play_songs_from_playlist(playlist_name)
        elif choice == '12':
            print("Exiting the service. Goodbye!")
            break
        else:
            print("Invalid option. Please try again.\n")

if __name__ == "__main__":
    main()



Music Streaming Service
1. Browse genres
2. Browse songs in a genre
3. Subscribe to a genre
4. View subscriptions
5. Discover music
6. Create playlist
7. Add song to playlist
8. Search for a song
9. Play a song
10. Play all songs in a genre
11. Play all songs from a playlist
12. Exit
Choose an option: 1

Available genres:
- acoustic
- afrobeat
- alt-rock
- alternative
- ambient
- anime
- black-metal
- bluegrass
- blues
- bossanova
- brazil
- breakbeat
- british
- cantopop
- chicago-house
- children
- chill
- classical
- club
- comedy
- country
- dance
- dancehall
- death-metal
- deep-house
- detroit-techno
- disco
- disney
- drum-and-bass
- dub
- dubstep
- edm
- electro
- electronic
- emo
- folk
- forro
- french
- funk
- garage
- german
- gospel
- goth
- grindcore
- groove
- grunge
- guitar
- happy
- hard-rock
- hardcore
- hardstyle
- heavy-metal
- hip-hop
- holidays
- honky-tonk
- house
- idm
- indian
- indie
- indie-pop
- industrial
- iranian
- j-dance
- j-idol
- j-pop
- j-rock
- ja

Github Link: https://github.com/EmmanFeBenito/SA1_FeBenito_2023029891